# Building Multimodal AI Applications with LangChain & the OpenAI API 

## Goals 

Videos can be full of useful information, but getting hold of that info can be slow, since you need to watch the whole thing or try skipping through it. It can be much faster to use a bot to ask questions about the contents of the transcript.

In this project, you'll download a tutorial video from YouTube, transcribe the audio, and create a simple Q&A bot to ask questions about the content.

### Maintenance note, May 2024

Since this code-along was released, the Python packages for working with the OpenAI API have changed their syntax. The instructions, hints, and code have been updated to use the latest syntax, but the video has not been updated. Consequently, it is now slightly out of sync. Trust the workbook, not the video.

- Understanding the building blocks of working with Multimodal AI projects
- Working with some of the fundamental concepts of LangChain  
- How to use the Whisper API to transcribe audio to text 
- How to combine both LangChain and Whisper API to create ask questions of any YouTube video 

## Before you begin

You'll need a developer account with [OpenAI ](https://auth0.openai.com/u/signup/identifier?state=hKFo2SAyeTZBU1pzbUNWYWs3Wml5OWVvUVh4enZldC1LYU9PMaFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIDFUakNoUGFMLUdNWFpfQkpqdncyZjVDQk9xUTE4U0xDo2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q) and a create API Key. The API secret key will be stored in your 'Environment Variables' on the side menu. See the *getting-started.ipynb* notebook for details on setting this up.

## Task 0: Setup

The project requires several packages that need to be installed into Workspace.

- `langchain` is a framework for developing generative AI applications.
- `yt_dlp` lets you download YouTube videos.
- `tiktoken` converts text into tokens.
- `docarray` makes it easier to work with multi-model data (in this case mixing audio and text).

### Instructions

Run the following code to install the packages.

In [7]:
# Install the openai package, locked to version 1.27
!pip install openai==1.27

# Install the langchain package, locked to version 0.1.19
!pip install langchain==0.1.19

# Install the langchain-openai package, locked to version 0.1.6
!pip install langchain-openai==0.1.6

# Install the yt_dlp package, locked to version 2024.4.9
!pip install yt_dlp==2024.4.9

# Install the tiktoken package, locked to version 0.6.0
!pip install tiktoken==0.6.0

# Install the docarray package, locked to version 0.40.0
!pip install docarray==0.40.0

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Instructions

## Task 1: Import The Required Libraries 

For this project we need the `os` and the `yt_dlp` packages to download the YouTube video of your choosing, convert it to an `.mp3` and save the file. We will also be using the `openai` package to make easy calls to the OpenAI models we will use. 

### Instructions

Import the following packages.

- Import `os`. 
- Import `glob`.
- Import `openai`.
- Import `yt_dlp` with the alias `youtube_dl`.
- From the `yt_dlp` package, import `DowloadError`.
- Assign `openai_api_key` to `os.getenv("OPENAI_API_KEY")`.

In [8]:
# Import the os package
import os

# Import the glob package
import glob

# Import the openai package 
import openai

# Import the yt_dlp package as youtube_dl
import yt_dlp as youtube_dl

# Import DownloadError from yt_dlp
from yt_dlp import DownloadError

# Import DocArray 
import docarray
openai_api_key= os.getenv("OPENAI_API_KEY")


We will also assign the variable `openai_api_key` to the environment variable "OPEN_AI_KEY". This will help keep our key secure and remove the need to write it in the code here. 

In [9]:
openai_api_key = os.getenv("OPENAI_API_KEY")

## Task 2: Download the YouTube Video

After creating the setup, the first step we will need to do is download the video from Youtube and convert it to an audio file (.mp3). 

We'll download a DataCamp tutorial about machine learning in Python.

We will do this by setting a variable to store the `youtube_url` and the `output_dir` that we want the file to be stored. 

The `yt_dlp` allows us to download and convert in a few steps but does require a few configuration steps. This code is provided to you. 

Lastly, we will create a loop that looks in the `output_dir` to find any .mp3 files. Then we will store those in a list called `audio_files` that will be used later to send each file to the Whisper model for transcription. 

### Instructions

- Run the code to set the URL of the video, `youtube_url`, the directory to store the downloaded video, `youtube_url`, and the download settings, `ydl_config`.

_This code can be adapted to any video you choose! Just change the URL._

In [10]:
# An example YouTube tutorial video
youtube_url = "https://www.youtube.com/watch?v=aqzxYofJ_ck"

# Directory to store the downloaded video
output_dir = "files/audio/"

# Config for youtube-dl
ydl_config = {
    "format": "bestaudio/best",
    "postprocessors": [
        {
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }
    ],
    "outtmpl": os.path.join(output_dir, "%(title)s.%(ext)s"),
    "verbose": True
}


### Instructions

- Check if `output_dir` exists, if not, then make that directory.
- Try to download the video using the specified configuration.
  -  If a DownloadError occurs, attempt to download the video again.

<details>
<summary>Code hints</summary>
<p>
    
- Check whether a directory exists using `os.path.exists()`.
- Make a new directory with `os.makedirs()`.

---
    
The pattern for trying to do something then handling an error is:
    
```py
try:
    # do something
except TypeOfError:
    # handle the error
```

---
    
The YouTube downloader code pattern is similar to opening a file.

```py
with youtube_dl.YoutubeDL(config_file) as ydl:
    ydl.download([video_url])
```

</p>
</details>

In [ ]:
# Check if the output directory exists, if not create it
if not os.path.exists(output_dir):
        os.makedirs(output_dir)

# Print a message indicating which video is being downloaded
print(f"Downloading video from {youtube_url}")

# Try to download the video using the specified configuration
# If a DownloadError occurs, attempt to download the video again
try:
    with youtube_dl.YoutubeDL(ydl_config) as ydl:
        ydl.download([youtube_url])
except DownloadError:
     with youtube_dl.YoutubeDL(ydl_config) as ydl: 
        ydl.download([youtube_url])

[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2024.04.09 from yt-dlp/yt-dlp [ff0779267] (pip) API
[debug] params: {'format': 'bestaudio/best', 'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}], 'outtmpl': 'files/audio/%(title)s.%(ext)s', 'verbose': True, 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.8.10 (CPython x86_64 64bit) - Linux-5.10.234-225.921.amzn2.x86_64-x86_64-with-glibc2.29 (OpenSSL 1.1.1f  31 Mar 2020, glibc 2.31)
[debug] exe versions: ffmpeg 4.2.7, ffprobe 4.2.7
[debug] Optional libraries: Cryptodome-3.23.0, brotli-1.1.0, certifi-2019.

[youtube] Extracting URL: https://www.youtube.com/watch?v=aqzxYofJ_ck
[youtube] aqzxYofJ_ck: Downloading webpage
[youtube] aqzxYofJ_ck: Downloading ios player API JSON
[youtube] aqzxYofJ_ck: Downloading android player API JSON


ERROR: [youtube] aqzxYofJ_ck: Sign in to confirm you’re not a bot. This helps protect our community. Learn more
  File "/home/repl/.local/lib/python3.8/site-packages/yt_dlp/extractor/common.py", line 734, in extract
    ie_result = self._real_extract(url)
  File "/home/repl/.local/lib/python3.8/site-packages/yt_dlp/extractor/youtube.py", line 4182, in _real_extract
    self.raise_no_formats(reason, expected=True)
  File "/home/repl/.local/lib/python3.8/site-packages/yt_dlp/extractor/common.py", line 1251, in raise_no_formats
    raise ExtractorError(msg, expected=expected, video_id=video_id)

[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2024.04.09 from yt-dlp/yt-dlp [ff0779267] (pip) API
[debug] params: {'format': 'bestaudio/best', 'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}], 'outtmpl': {'default': 'files/audio/%(title)s.%(

[youtube] Extracting URL: https://www.youtube.com/watch?v=aqzxYofJ_ck
[youtube] aqzxYofJ_ck: Downloading webpage
[youtube] aqzxYofJ_ck: Downloading ios player API JSON
[youtube] aqzxYofJ_ck: Downloading android player API JSON


ERROR: [youtube] aqzxYofJ_ck: Sign in to confirm you’re not a bot. This helps protect our community. Learn more
  File "/home/repl/.local/lib/python3.8/site-packages/yt_dlp/extractor/common.py", line 734, in extract
    ie_result = self._real_extract(url)
  File "/home/repl/.local/lib/python3.8/site-packages/yt_dlp/extractor/youtube.py", line 4182, in _real_extract
    self.raise_no_formats(reason, expected=True)
  File "/home/repl/.local/lib/python3.8/site-packages/yt_dlp/extractor/common.py", line 1251, in raise_no_formats
    raise ExtractorError(msg, expected=expected, video_id=video_id)



DownloadError: ERROR: [youtube] aqzxYofJ_ck: Sign in to confirm you’re not a bot. This helps protect our community. Learn more

To find the audio files that we will use the `glob`module that looks in the `output_dir` to find any .mp3 files. Then we will append the file to a list called `audio_files`. This will be used later to send each file to the Whisper model for transcription. 

### Instructions

Find the audio file in the output directory.

- Find all the MP3 audio files in the output directory by joining the output directory to the pattern `*.mp3` and using glob to list them.
- Select the first file in the list and assign it to `audio_filename`.
- _Check your work._ Print `audio_filename`.

In [19]:
# Find the audio file in the output directory

# Find all the audio files in the output directory
audio=glob.glob(os.path.join(output_dir,"*.mp3"))

# Select the first audio file in the list
audio_filename=audio[0]

# Print the name of the selected audio file
print(audio_filename)

files/audio/Databytes.mp3


## Task 3: Transcribe the Video using Whisper

In this step we will take the downloaded and converted Youtube video and send it to the Whisper model to be transcribed. To do this we will create variables for the `audio_file`, for the `output_file` and the model. 

Using these variables we will:
- create a list to store the transcripts
- Read the Audio File 
- Send the file to the Whisper Model using the OpenAI package 

### Instructions

Transcribe the audio file.

- _The audio file, output file, and model are definied for you._
- Define an OpenAI client model. Assign to client.
- Open the audio file as read-binary (`"rb"`).
  - Use the Whisper model to create a transcription of the opened audio file. Assign to `response`.
- Extract the transcript from the response.

<details>
<summary>Code hints</summary>
<p>
 
Define the client model with `openai.OpenAI()`.

---

The code pattern for opening a file in read-binary mode is.
    
```py
with open(file, "rb") as file_handle:
    # do something
```

---
    
To use the OpenAI API to create transcription, call the client's `.audio.transcriptions.create()` method, passing the file and the model.

```py
response = client.audio.transcriptions.create(file=audio, model=model)
```

---
    
The transcript text is in the `.text` element of the response.
    
</p>
</details>

In [21]:
# Use these settings
audio_file = audio_filename
output_file = "files/transcripts/transcript.txt"
model = "whisper-1"

# Transcribe the audio file to text using OpenAI API
print("converting audio to text...")

# Define an OpenAI client model. Assign to client.
client = openai.OpenAI()

# Open the audio file as read-binary
with open(audio_file,"rb") as f:
    # Use the model to create a transcription
    resp = client.audio.transcriptions.create(file = f , model = model)

# Extract the transcript from the response
trans=resp.text

# Print the transcript
print(trans)

converting audio to text...
Hi, in this tutorial, we're going to look at a data pre-processing technique for machine learning called splitting your data. That is splitting your data set into a training set and a testing set. Now, before we get to the code, you might wonder, why do I need to do this? And really, there are going to be two problems if you don't. So if you train your machine learning model on your whole data set, then you've not tested the model on anything else. And that means you don't know how well your model is going to perform on other data sets. Secondly, it's actually even worse than this, because you risk overfitting the model. And that means that you've made your model work really well for one data set, but that gives a cost of model performance on other data sets. So not only do you not know how well the model is going to perform on other data sets, it's probably going to be worse than it could be. So you might also wonder when in your machine learning workflow, 

### Instructions

Save the transcript to a text file.

- If the directory for the output file doesn't exist, make it.
- Write the transcript to the output file

<details>
<summary>Code hints</summary>
<p>
 
- Get the directory to write to with `os.path.dirname()` and the output file.
- Make a directory with `os.makedirs()`. Set `exist_ok` to `True` to prevent errors if the directory already exists.

---

The code pattern for writing a text file is as follows.
    
```py
with open(filename, "w") as file:
    file.write(text)
```


</p>
</details>

In [24]:
# Create the directory for the output file if it doesn't exist
os.makedirs(os.path.dirname(output_file),exist_ok=True)
# Write the transcript to the output file
with open(output_file,"w") as file:
    file.write(trans)



## Task 4: Create a TextLoader using LangChain 

In order to use text or other types of data with LangChain we must first convert that data into Documents. This is done by using loaders. In this tutorial, we will use the `TextLoader` that will take the text from our transcript and load it into a document. 

### Instructions

Load the documents from the text file using a TextLoader.

- From the `langchain.document_loaders` module, import `TextLoader`.
- Create a `TextLoader`, passing it the directory of the transcripts, `"./files/text"`. Assign to `loader`.
- Use the TextLoader to load the documents. Assign to `docs`.

In [27]:
# From the langchain.document_loaders module, import TextLoader
from langchain.document_loaders import TextLoader

# Create a `TextLoader`, passing the directory of the transcripts. Assign to `loader`.
loader=TextLoader("./files/transcripts/transcript.txt")

# Use the TextLoader to load the documents. Assign to docs.
docs=loader.load()

In [28]:
# Show the first element of docs to verify it has been loaded 
docs[0]

Document(page_content="Hi, in this tutorial, we're going to look at a data pre-processing technique for machine learning called splitting your data. That is splitting your data set into a training set and a testing set. Now, before we get to the code, you might wonder, why do I need to do this? And really, there are going to be two problems if you don't. So if you train your machine learning model on your whole data set, then you've not tested the model on anything else. And that means you don't know how well your model is going to perform on other data sets. Secondly, it's actually even worse than this, because you risk overfitting the model. And that means that you've made your model work really well for one data set, but that gives a cost of model performance on other data sets. So not only do you not know how well the model is going to perform on other data sets, it's probably going to be worse than it could be. So you might also wonder when in your machine learning workflow, as yo

## Task 5: Create an In-Memory Vector Store 

Now that we have created Documents of the transcription, we will store that Document in a vector store. Vector stores allows LLMs to traverse through data to find similiarity between different data based on their distance in space. 

For large amounts of data, it is best to use a designated Vector Database. Since we are only using one transcript for this tutorial, we can create an in-memory vector store using the `docarray` package. 

We will also tokenize our queries using the `tiktoken` package. This means that our query will be seperated into smaller parts either by phrases, words or characters. Each of these parts are assigned a token which helps the model "understand" the text and relationships with other tokens. 

### Instructions

- Import the `tiktoken` package. 

In [29]:
# Import the tiktoken package
import tiktoken

## Task 6: Create the Document Search 

We will now use LangChain to complete some important operations to create the Question and Answer experience. Let´s import the follwing: 

- Import `RetrievalQA` from `langchain.chains` - this chain first retrieves documents from an assigned Retriver and then runs a QA chain for answering over those documents 
- Import `ChatOpenAI` from `langchain.chat_models` - this imports the ChatOpenAI model that we will use to query the data 
- Import `DocArrayInMemorySearch` from `langchain.vectorstores` - this gives the ability to search over the vector store we have created. 
- Import `OpenAIEmbeddings` from `langchain.embeddings` - this will create embeddings for the data store in the vector store. 
- Import `display` and `Markdown`from `IPython.display` - this will create formatted responses to the queries. (

In [36]:
# From the langchain.chains module, import RetrievalQA
from langchain.chains import RetrievalQA

# From the langchain_openai package, import ChatOpenAI, OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# From the langchain.vectorstores module, import DocArrayInMemorySearch
from langchain.vectorstores import DocArrayInMemorySearch

Now we will create a vector store that will use the `DocArrayInMemory` search methods which will search through the created embeddings created by the OpenAI Embeddings function. 

### Instructions

Create an in-memory search object from the specified documents, `docs` and embeddings, `OpenAIEmbeddings()`. Assign to `db`.

<details>
<summary>Code hints</summary>
<p>
 
- `DocArrayInMemorySearch` has a `.from_documents()` method that takes two arguments: the documents and the embeddings.
- The documents were created when you used the `TextLoader`.
- `OpenAIEmbeddings()` specifies the type of embeddings to use.

</p>
</details>

In [37]:
# Create a new DocArrayInMemorySearch instance from the specified documents and embeddings
db = DocArrayInMemorySearch.from_documents(docs,OpenAIEmbeddings())

We will now create a retriever from the `db` we created in the last step. This enables the retrieval of the stored embeddings. Since we are also using the `ChatOpenAI` model, will assigned that as our LLM.

Recall that the temperature of an LLM refers to how random the results are. Setting the temperature to zero makes the results more repeatable.

### Instructions

- Convert the `DocArrayInMemorySearch` instance to a retriever. Assign to `retriever`.
- Create a new `ChatOpenAI` instance with a temperature of `0.0`. Assign to `llm`. 

<details>
<summary>Code hints</summary>
<p>
 
- `DocArrayInMemorySearch` has a `.as_retriever()` method to convert it to a retriever object. No arguments are required.
- Create a client model with `ChatOpenAI()`, setting the `temperature` argument.

</p>
</details>

In [38]:
# Convert the DocArrayInMemorySearch instance to a retriever
retriever= db.as_retriever()

# Create a new ChatOpenAI instance with a temperature of 0.0
llm= ChatOpenAI(temperature=0.0)

Our last step before starting to ask questions is to create the `RetrievalQA` chain. This chain takes in the:  
- The `llm` we want to use.
- The `chain_type` which is how the model retrieves the data. Here we will use a _stuff_ chain, where all the documents are stuffed into the prompt. It is the simplest type, but only works where you only have a few small documents.
- The `retriever` that we have created.
- An option called `verbose` that prints details of each step of the chain.

### Instructions

Create a new RetrievalQA instance from the chain type. 

- Set `llm` to the ChatOpenAI instance you just created.
- Set `chain_type` to `"stuff"`.
- Set `retriever` to the retriever you just created.
- Set `verbose` to `True`.

<details>
<summary>Code hints</summary>
<p>
 
The code pattern to create a `RetrievalQA` instance from a chain type is as follows.
    
```py
RetrievalQA.from_chain_type(
    llm=your_model,
    chain_type=your_chain_type,
    retriever=your_retriever,
    any_other_options
)
```

</p>
</details>

In [40]:
# Create a new RetrievalQA instance with the specified parameters
qa_stuff= RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever= retriever,
    verbose= True
)
    # The ChatOpenAI instance to use for generating responses
    # The type of chain to use for the QA system
    # The retriever to use for retrieving relevant documents
    # Whether to print verbose output during retrieval and generation


## Task 7: Create the Queries 

Now we are ready to create queries about the YouTube video and read the responses from the LLM. This done first by creating a query and then running the RetrievalQA we setup in the last step and passing it the query. 

### Instructions

Ask GPT some questions about the transcript.

- Create question, `"What is this tutorial about?"`. Assign to `query`.
- Invoke the query through the RetrievalQA instance. Assign to `response`. 
- Print the response.

<details>
<summary>Code hints</summary>
<p>
 
Call the `.invoke()` method of the RetrievalQA instance, passing the query. The code pattern is as follows.
    
```py
response = qa.invoke(text)
```

</p>
</details>

In [42]:
# Set the query to be used for the QA system
query="what is the tuorial about?"

# Invoke the query through the RetrievalQA instance. Assign to response.
response = qa_stuff.run(query)

# Print the response to the console
response




> Entering new RetrievalQA chain...

> Finished chain.


"The tutorial is about a data pre-processing technique for machine learning called splitting your data. It focuses on splitting a data set into a training set and a testing set to avoid overfitting and to ensure the model's performance on unseen data sets. The tutorial also covers the importance of when to split the data in the machine learning workflow and provides a practical example using loan application data."

We can continue on creating queries and even creating queries that we know would not be answered in this video to see how the model responds. 

In [44]:
# Set the query to be used for the QA system
query = "What is the difference between a training set and test set?"

# Invoke the query through the RetrievalQA instance and store the response
response=qa_stuff.run(query)

# Print the response to the console
response



> Entering new RetrievalQA chain...

> Finished chain.


'The training set is used to train the machine learning model, meaning the model learns patterns and relationships within the data. The test set, on the other hand, is used to evaluate the performance of the trained model on unseen data. The training set helps the model learn, while the test set helps assess how well the model generalizes to new data.'

In [45]:
# Set the query to be used for the QA system
query = "Who should watch this lesson?"

# Invoke the query through the RetrievalQA instance and store the response
response=qa_stuff.run(query)

# Print the response to the console
response



> Entering new RetrievalQA chain...

> Finished chain.


'This lesson on splitting data into training and testing sets is beneficial for individuals who are learning or working with machine learning techniques. It provides essential knowledge on the importance of splitting data, the risks of not doing so, and the proper timing of this process in a machine learning workflow.'

In [46]:
# Set the query to be used for the QA system
query = "Who is the greatest football team on earth?"

# Invoke the query through the RetrievalQA instance and store the response
response=qa_stuff.run(query)

# Print the response to the console
response



> Entering new RetrievalQA chain...

> Finished chain.


"I don't know the answer to that question."

In [47]:
# Set the query to be used for the QA system
query = "How long is the circumference of the earth?"

# Invoke the query through the RetrievalQA instance and store the response
response=qa_stuff.run(query)

# Print the response to the console
response



> Entering new RetrievalQA chain...

> Finished chain.


"I don't know the exact length of the circumference of the earth."

## All done, congrats! 